In [82]:
import pandas as pd
#%matplotlib notebook
#import seaborn as sns

df_ALL = pd.read_sas("./HN16_ALL.sas7bdat", format = 'sas7bdat', encoding='iso-8859-1')
df_ffq = pd.read_sas("./hn16_ffq.sas7bdat", format = 'sas7bdat', encoding='iso-8859-1')

# 칼럼 저장
# df_ffq.to_csv("df_ffq.csv", encoding='iso-8859-1')
# df_data.to_csv("df_data.csv", encoding='iso-8859-1')

### 데이터 준비
1. 2개의 데이터프레임에서 중복된 칼럼 확인
2. 2번째 데이터프레임에서 키값으로 사용할 ID를 제외한 나머지 중복 칼럼을 삭제
3. ID를 키값으로 2개의 데이터 프레임 머지(left, right, inner, outer, index 등 세팅 확인
4. 정상적으로 데이터프레임 합쳐졌는지, 널값으로 확인
5. 이후 decision tree sklearn classifier를 이용해서 확인(코딩)

In [83]:
# 2개의 리스트에 각각의 데이터프레임의 컬럼명을 저장
list_col_all = df_ALL.columns.values
list_col_ffq = df_ffq.columns.values

# 리스트를 집합으로 변환하여 intersection 사용하여 교집합 구하고 다시 리스트로 반환
list_col_overlab = list(set(list_col_all).intersection(list_col_ffq))
# print(len(list_col_overlab)) # 갯수 확인
# list_col_overlab # 컬럼 확인

In [84]:
# 총 26개의 중복 칼럼명 리스트에서 merge 시 key값으로 사용할 'ID'' 삭제
list_col_overlab.remove('ID')
# print(len(list_col_overlab)) # 갯수 확인
# list_col_overlab # 컬럼 확인

In [85]:
# df_ffq 데이터프레임에서 ID를 제외한 나머지 중복 컬럼 삭제
print(df_ffq.shape)
df_ffq.drop(list_col_overlab, axis=1, inplace=True)
print(df_ffq.shape)

(8150, 434)
(8150, 409)


In [86]:
# 데이터프레임 merge
df_merge = pd.merge(df_ALL, df_ffq, how='inner', on=[
                                  'ID' # 개인 아이디
                                  
                                 ])
print(df_merge.shape)

(8150, 1176)


In [87]:
# merge 데이터프레임 확인
# df_merge.to_csv("df_merge.csv", encoding='iso-8859-1')

In [88]:
# 19세 이상으로만 제한
df_data = df_merge.loc[(18 < df_merge.age), :]

# 혈압치료중인 대상 제외
df_data = df_data.loc[(1 < df_data.DI1_pt) | (df_data.DI1_pt < 1), :]

df_data.shape

(4727, 1176)

In [77]:
# NaN값 컬럼별 카운트 확인
#df_nan_sort = df_data.isnull().sum()

In [78]:
# null값 많은 컬럼 및 카운트 저장
#df_nan_sort.to_csv("df_nan_sort.csv", encoding='iso-8859-1')

In [89]:
# 모든 값이 NaN값이 컬럼 삭제
df_data = df_data.dropna(axis=1, how='all')
df_data.shape

(4727, 1122)

In [90]:
# 고혈압 유병여부 NaN값 대상 삭제
df_data = df_data.dropna(subset=['HE_HP'])
df_data.shape

(4708, 1122)

In [91]:
# index 재 생성을 위해 ID로 인덱스 생성 후 인덱스 삭제
df_data =  df_data.set_index('ID')
df_data = df_data.reset_index()


In [96]:
# 고혈압 유병여부
# 1. 정상
# 2. 고혈압전단계
# 3. 고혈압
# classification을 위해3. 고혈압을 2.로 변경하여 이산형으로 수정

df_data.loc[df_data.HE_HP == 3.0, 'HE_HP'] = 2.0
print("HE_HP :\n")
print(pd.value_counts(df_data.HE_HP.values, sort=False).sort_index()) # 고혈압 유병 여부

HE_HP :

1.0    2640
2.0    2068
dtype: int64


In [141]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn import tree
import graphviz
from tqdm import tqdm_notebook

#x_name = ['age', 'sex']
# x_name = ['HE_ht']
# y_name = 'HE_HP'

def saveDTR(input_df, x_name, y_name) :

    arrX1 = df_data[[x_name]]
    arrY = df_data[y_name]

    arrX1 = arrX1.fillna(-1)
    #print(arrX1.isnull().sum())

    model = DecisionTreeClassifier(
        criterion = 'gini',
        max_depth=2
    )
    #print(model)

    model.fit(arrX1,arrY)

    dot_data = tree.export_graphviz(model, out_file=None, feature_names=[x_name], class_names=['정상', '고혈압군'])

    graph = graphviz.Source(dot_data) 
    graph.render("./pdf/%s+%s" % (x_name,y_name))
    #graph.render("%s+%s" % (x_name,y_name))

    modelPrediction = model.predict(arrX1)
    accuracyRate = accuracy_score(
        y_true=arrY
    ,   y_pred=modelPrediction
    )
    accuracyRate_list.append([x_name, y_name, accuracyRate])

    #print('정확도 = ', accuracyRate)

# HE_HP(고혈압 유병여부) 제외
skip_list = [
    df_data.HE_HP.name
    #, df_data.ID.name
    
]

# accuracyRate 담을 변수 준비
accuracyRate_list = []

for x_name in tqdm_notebook(df_data.columns):
    if x_name in skip_list : continue
    #print(df_data[x_name].dtype.name)
    if df_data[x_name].dtype == 'object' : continue
    saveDTR(df_data, x_name, df_data.HE_HP.name)
    
# accuracyRate_list > dataframe으로 변환해서 csv파일로 저장   
df_accuracyRate = pd.DataFrame(accuracyRate_list)
df_accuracyRate.to_csv("accuracyRate_list.csv")